## Product Description Clustering with TF-IDF, K-Means, and LSA

In this notebook, I use natural language processing, K-Means clustering, and latent semantic analysis (LSA) to cluster and plot 3,440 effects pedals from [Reverb.com](https://reverb.com/). The steps below summarize what this notebook encompasses.

1. Read in CSV file, remove duplicate pedals, and clean product descriptions using Pandas
2. Tokenize and stem product descriptions using NLTK
3. Transform stemmed descriptions into a matrix using scikit-learns's TF-IDF vectorizer
4. Cluster TF-IDF matrix using K-Means clustering
5. Perform dimension reduction using latent semantic analysis (LSA)
6. Plot description clusters on a 3-dimensional axis using Plotly

## Getting Started

Below are the required packages for this notebook. The notebook is also written using Python 3.5. 

In [1]:
import re
import pandas as pd
import collections
import numpy as np
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
init_notebook_mode(connected=True)

The effects pedal data was collected into a CSV file using a different script and Reverb.com's listing API. The file contains 6,650 pedals and 32 columns.

In [2]:
rdf = pd.read_csv('reverb_effects_pedals_v4_08032016.csv')
rdf.shape

(6650, 32)

For this project, I tried to remove as many pedals as possible to avoid clusters being created for a single pedal type. To remove these duplicates, I use the `drop_duplicates` method on the following columns, while keeping the first instance of each duplicate. This method isn't perfect, and there are still duplicates, but it removes the large majority of them.

* web_url
* title
* description
* id
* model

In [3]:
rdf = rdf.drop_duplicates('web_url', keep='first')

In [4]:
rdf.title = rdf.title.str.lower().str.strip()
rdf = rdf.drop_duplicates('title', keep='first')

In [5]:
rdf.description = rdf.description.str.lower().str.strip()
rdf = rdf.drop_duplicates('description', keep='first')

In [6]:
rdf = rdf.drop_duplicates('id', keep='first')

In [7]:
rdf = rdf.drop_duplicates('model', keep='first')

This leaves 3,440 pedals in the DataFrame.

In [8]:
rdf.shape

(3440, 32)

## Cleaning and Tokenizing Product Descriptions

Now that the majority of duplicate pedals have been removed, I filter out brand names from each product description. I did this to avoid them leading towards any bias in the TF-IDF portion of the project. 

In [9]:
descriptions = rdf.description.str.lower().str.strip()

In [10]:
def filter_brands(descriptions, brand_names):
    clean_descriptions = []
    for description in descriptions:
        for brand in brand_names:
            if brand in description:
                description = description.replace(brand, '').strip()
            else:
                pass
        clean_descriptions.append(description)
    return clean_descriptions

In [11]:
filtered_descriptions = filter_brands(descriptions, set(rdf.make.str.lower()))

After collecting all of the filtered descriptions, I use [NLTK](http://www.nltk.org/), which is a natural language processing Python library to both stem and tokenize words of each description. Stemming words helps account for different forms of a word, such as analyze, analyzing, and analyzed. Tokenization splits text into meaningful elements, such as words, punctuation and symbols and treats each as a separate entity. 

To avoid including numbers, punctuation, or any symbols, I use regex to match words that contain letters only. 

In [12]:
def stem_and_tokenize(description):
    stemmer = SnowballStemmer('english')
    tokens = [word.lower().strip() for sentence in sent_tokenize(description) for word in word_tokenize(sentence)]
    filtered_tokens = [token for token in tokens if re.search('^[A-Za-z]+$', token)]
    return [stemmer.stem(token) for token in filtered_tokens]

In [13]:
def tokenize(description):
    tokens = [word.lower().strip() for sentence in sent_tokenize(description) for word in word_tokenize(sentence)]
    filtered_tokens = [token for token in tokens if re.search('^[A-Za-z]+$', token)]
    return filtered_tokens

In [14]:
def generate_stems(descriptions):
    return [stem_and_tokenize(description) for description in descriptions]

In [15]:
def generate_tokens(descriptions):
    return [tokenize(description) for description in descriptions]

In [16]:
token_lists = generate_tokens(filtered_descriptions)
stem_lists = generate_stems(filtered_descriptions)

Below shows the first 10 elements of the first description from both the stemmed description and tokenized description lists. Notice how the words `specifically` and `accomodate` are stemmed in the first list.

In [17]:
print('First 10 stems: ', stem_lists[0][:10])
print('First 10 tokens:', token_lists[0][:10])

First 10 stems:  ['this', 'great', 'chorus', 'pedal', 'is', 'design', 'specif', 'to', 'accommod', 'the']
First 10 tokens: ['this', 'great', 'chorus', 'pedal', 'is', 'designed', 'specifically', 'to', 'accommodate', 'the']


To keep track of each stem and token pair, I create a DataFrame to account for them. This will come in use later when I retrieve the original words in a description for cluster categorization. The key to being able to access the actual word later is using the stemmed word as the DataFrame index, since the stemmed words are what will be used for clustering. This was inspired by Brandon Rose's [document clustering notebook](http://brandonrose.org/clustering).

In [18]:
word_df = pd.DataFrame([x for lst in token_lists for x in lst], 
                       [y for lst in stem_lists for y in lst])
word_df.columns = ['word']
word_df.head(10)

,word
this,this
great,great
chorus,chorus
pedal,pedal
is,is
design,designed
specif,specifically
to,to
accommod,accommodate
the,the


All final tokens and stems are combined into a single list called `final_tokens` and `final_stems` below using list comprehensions. Example descriptions have been printed out for reference.

In [19]:
final_tokens = [' '.join([word for word in lst]) for lst in token_lists]
final_stems = [' '.join([word for word in lst]) for lst in stem_lists]
print('Final Stem Example: ', final_stems[0])
print()
print('Final Token Example: ' , final_tokens[0])

Final Stem Example:  this great chorus pedal is design specif to accommod the rang and applic for the electr bass to avoid muddi the effect frequenc rang can be accur control use the low filter knob featur specif design for bass applic low filter knob to avoid muddi stereo or mono output from subtl detun to rich spacious chorus ac adaptor or batteri oper five year warranti

Final Token Example:  this great chorus pedal is designed specifically to accommodate the range and applications for the electric bass to avoid muddiness the effected frequency range can be accurately controlled using the low filter knob features specifically designed for bass applications low filter knob to avoid muddiness stereo or mono outputs from subtle detuning to rich spacious chorus ac adaptor or battery operation five year warranty


## Transforming the Descriptions Into a TF-IDF Matrix

In [20]:
%%html
<style>
table {float:left}
</style>

TF-IDF stands for term frequency-inverse document frequency. This metric is used as a weight to deterimine how important a word is to a single document included in a collection of documents. As the frequency of a term increases in a single document, it's term frequency increases. It is then offset by the inverse document frequency (IDF), which is a logarithmic-based fraction of the number of total documents divided by the number of times a word appears in that group of documents. More information on this metric can be found [here](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

For this project, I use [scikit-learn's `TfidfVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) object to both fit and transform the descriptions into a matrix, which extracts features from each description. Below are the parameters I set and a description of each.

| Parameters | Description |       
| --- | ------------- |
| stop_words | Common words used in a language that will most likely appear in the majority of documents. These words will be removed from final token matrix. | 
| tokenizer | Used to override the default tokenization method. |
| max_df | Removes any tokens seen in the set percentage or more of the document collection. |

After fitting and transforming the descriptions, we have a matrix with 3,440 rows and 9,702 columns. The 3,440 rows represent each description, while each column reflects each word.

In [21]:
tfidf = TfidfVectorizer(stop_words='english', tokenizer=stem_and_tokenize, max_df=0.8)
tfidf_matrix = tfidf.fit_transform(final_stems)
tfidf_matrix.shape

(3440, 9701)

To get an idea of what this matrix looks like visually, I convert it to a Pandas DataFrame and print out its first 10 rows. Most cells are filled with zeros because each description includes a very small sample of words compared to the overall collection of words in all of the descriptions. Thus, the majority of words in the description list will have a 0 term frequency in each description which results in a TF-IDF weight of 0. This type of matrix is also known as a [sparse matrix](https://en.wikipedia.org/wiki/Sparse_matrix).

In [22]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray())
tfidf_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,9691,9692,9693,9694,9695,9696,9697,9698,9699,9700
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Clustering with K-Means

#### Overview
K-Means clustering is a unsupervised machine learning algorithm that clusters a group of data points together around a pre-defined number of centroids. After the initial centroids are set and each data point is assigned to a cluster, the clusters' new center points are calculated. This continues iteratively until a small number of points are reassigned to different groups. Reaching this point is also called convergence.   

#### Defining the Number of Clusters

There are several ways to calculate the best number of clusters for the dataset. Below are two common methods for doing so.

| Method | Description |       
| --- | ------------- |
| Elbow Method | Plot the sum of squared errors (SSE) for each cluster amount, and choose the k-value at the "elbow" of the plot.| 
| Silhouette Method | Measures how well a data point is correlated to the cluster it's in and it's neighbouring cluster. A score closer to 1 signals that a data point is in the correct cluster.  |

Due to the number of descriptions in this dataset, it would be very time intensive to run K-Means on every possible number of clusters. Because of this, I decided to manually choose the number of clusters based on my intuition on how I thought they may be clustered. I thought that clusters would be grouped by type of pedal, such as delay, reverb, distortion, and so on. I ended up with 15 clusters based on this assumption.

In [23]:
num_clusters = 15
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=15, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

To double check that the number of cluster labels is equal to the number of descriptions after fitting, I print out the length of the number of labels converted to a list. It matches the total number of descriptions.

In [24]:
cluster_labels = km.labels_.tolist()
print('# of labels: ', len(cluster_labels))

# of labels:  3440


The number of centroids equals 15 also, which can be seen below.

In [25]:
sorted_km_centroids = km.cluster_centers_.argsort()[:, ::-1]
len(sorted_km_centroids)

15

Next, I create a centroid DataFrame, which contains the centroid number, and it's top 10 features, which are top 10 words in the cluster from the TF-IDF matrix. A few examples can be seen below. As a reminder, these are the stemmed versions of the words from the filtered descriptions. The function `create_centroid_df` was inspired by this [document clustering tutorial](http://scikit-learn.org/stable/auto_examples/text/document_clustering.html). 

In [26]:
terms = tfidf.get_feature_names()
terms[:10]

['aa',
 'aaa',
 'ab',
 'abandon',
 'abbott',
 'abi',
 'abil',
 'abl',
 'ableton',
 'abnorm']

In [27]:
def create_centroid_df(sorted_km_centroids, num_clusters, feature_names):
    all_centroid_dicts = []
    for i in range(num_clusters):
        centroid_dict = dict(label = i)
        word_list = []
        for skc in sorted_km_centroids[i, :10]:
            token = list(word_df.ix[feature_names[skc]].word)[0]
            word_list.append(str(token))
            centroid_dict['words'] = ','.join(word_list)
        all_centroid_dicts.append(centroid_dict)
    return all_centroid_dicts

This DataFrame will be used later to help map each cluster's top 10 words to their correlated effects pedal.

In [28]:
all_centroid_dicts = create_centroid_df(sorted_km_centroids, num_clusters, terms)
all_centroid_dicts_df = pd.DataFrame(all_centroid_dicts)
all_centroid_dicts_df.head()

,label,words
0,0,"condition,excellent,good,working,mint,velcro,b..."
1,1,"pitch,whammy,shifting,harmonies,detuning,effec..."
2,2,"reverb,spring,hall,decay,room,plate,effected,s..."
3,3,"monaural,ohm,imp,jacks,minus,small,d,h,metal,a..."
4,4,"includedowners,includedoriginal,includedcondit..."


## Dimensionality Reduction Using Latent Semantic Analysis (LSA)

Latent semantic analysis helps find concepts in a collection of documents and maps similar documents together using Singular Value Decomposition. From a high-level, SVD helps reduce the number of dimensions in a matrix by selecting several vectors that are strong representations of the matrix overall. For this portion of the project, I want to plot each effects pedal on a 3-dimensional plane, so I will need LSA ot reduce the TF-IDF matrix to 3 features. 

In [29]:
lsa = TruncatedSVD(n_components=3, random_state=42)
lsa_matrix = lsa.fit_transform(tfidf_matrix)

In [30]:
x_values = lsa_matrix[:,0]
y_values = lsa_matrix[:,1]
z_values = lsa_matrix[:,2]

In [31]:
plot_df = pd.DataFrame(dict(x=x_values, y=y_values, z=z_values, title=rdf.title, label=cluster_labels))

Next, I map the `all_centroid_dicts_df` words to the labels in the `plot_df` DataFrame. I do this by using the `label` column as the index to match the same column in the `plot_df` DataFrame. The first 5 rows of the final DataFrame can be seen below.

In [32]:
plot_df['words'] = plot_df.label.map(all_centroid_dicts_df.set_index('label').words)
plot_df.head()

,label,title,x,y,z,words
0,12,boss ceb-1 bass chorus,0.210333,0.022681,0.004905,"chorus,flanger,effected,vibrato,toneprint,soun..."
1,12,boss ceb-3 bass chorus,0.233458,0.079280,-0.018021,"chorus,flanger,effected,vibrato,toneprint,soun..."
2,8,boss re-20 space echo,0.249691,0.173567,-0.162240,"delay,echo,analog,time,repeat,modulate,control..."
3,8,boss dm-2w waza craft analog delay - vintage r...,0.312848,0.331523,-0.209315,"delay,echo,analog,time,repeat,modulate,control..."
4,5,boss roland ds1 distortion guitar effects pedal,0.218231,-0.067033,-0.001687,"overdrive,distortion,tube,tone,screamer,pedal,..."


## Plotting the Clusters

The final step is visualizing the clusters. I use [Plotly offline](https://plot.ly/python/offline/) to do this with their `Scatter3d` object. Plotly makes it simple to create customized plots with interactivity for users. Several clusters can easily be pinpointed, which have been outlined below. 

| Cluster | Color | Location | Words |         
| ------- | ----- | -------- | ----- |
| Mooer Pedals | Dark Blue | Top | monaural,ohm,imp,jacks,minus,small |
| Delays & Echos | Green | Right | delay,echo,analog,time,repeat |
| Fuzz | Red | Left | fuzz,face,tone,germanium,mini | 
| Boosts, Overdrives & Distortions | Turquoise | Left-Center |  boost,pedal,gain,tone,amps |


The location of each pedal is based on data from the LSA matrix, while the color is based on the K-Means clustering model. This makes it easy to see where clusters could possibly be improved on. This particular plot shows that there is some room for improvement. Several pedals appear mislabeled, especially near the point of origin where several clusters appear mixed together. As a follow-up, I may try to figure out ways to clean those clusters up a bit.

In [33]:
data = [Scatter3d(
        x = plot_df.x,
        y = plot_df.y,
        z = plot_df.z,
        mode = 'markers',
        name = 'plot clusters',
        text = plot_df.title,
        showlegend = True,
        marker = dict(
            size = 5,
            color = plot_df.label,
            colorscale = 'Rainbow',
            showscale = True
        )
    )]
layout = Layout(
    title = 'Pedal Clusters based on Product Descriptions',
    margin = dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    xaxis = dict(
        showgrid = False,
        showticklabels = False,
        showline = False,
        zeroline = False
    ),
    yaxis = dict(
        showgrid = False,
        showticklabels = False,
        showline = False,
        zeroline = False
    ),
)
fig = Figure(layout=layout, data=data)
iplot(fig)